In [2]:
import spacy
import requests
import zipfile
import io
import polars as pl

In [3]:
data_url = "https://foerderdatenbankdump.fra1.cdn.digitaloceanspaces.com/data/data.zip"

In [4]:
response = requests.get(data_url)
response.raise_for_status()

with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    file_name = z.namelist()[0]
    with z.open(file_name) as f:
        df = pl.read_parquet(f)

print(f"{len(df)} Förderprogramme")
print(df.columns)
print(df.dtypes)

2488 Förderprogramme
['id_hash', 'id_url', 'url', 'title', 'description', 'more_info', 'legal_basis', 'contact_info_institution', 'contact_info_street', 'contact_info_city', 'contact_info_fax', 'contact_info_phone', 'contact_info_email', 'contact_info_website', 'funding_type', 'funding_area', 'funding_location', 'eligible_applicants', 'funding_body', 'further_links', 'checksum', 'license_info', 'previous_update_dates', 'last_updated', 'deleted']
[String, String, String, String, String, String, String, String, String, String, String, String, String, String, List(String), List(String), List(String), List(String), String, List(String), String, String, List(Datetime(time_unit='us', time_zone=None)), Datetime(time_unit='us', time_zone=None), Boolean]


In [23]:
filtered_df = df.filter(~pl.col("legal_basis").is_null())

In [24]:
filtered_df = filtered_df.sample(40)

In [25]:
filtered_df[["id_hash","legal_basis"]].write_csv("data/sample.csv")